### functions for error plotting

For error plotting the output files of BigStitcher are used

1. tpId_0_viewSetupId_>x<.>interest_point_name<.ip.txt : coordinates of all beads of one Set-up (one tile in one channel)
2. dataset.xml : files gives all the transformation matrixes to calculate the coordinates after transformation
3. tpId_0_viewSetupId_>x<.>interest_point_name<.corr.txt : corresponding beads of one Set-up (gives reference Set-up number, reference bead number)

<b>1</b> and <b>2</b> are used to calculate global coordinates after transformation

Global coordinates are matched to corresponding beads in <b>3</b> and distance is calculated 


<b>Input</b> are two lists of setupids: 
1. first list is one side with usually four tiles (four Set-up Ids)
2. second list is usually 2 or 3 sides (more than four Set-up Ids)

For each Set-up from the first list is compared to all the Set-ups from the second list


<b>Output</b> is a dataframe with following format:

<b>setupid 	corresponding_setupid 	beadid 	c 	c-norm 	x 	y 	z </b>

<b>setupid </b>: Set-up Id of Set-ups which are compared to each other "frist Set-up"<br>
<b>corresponding_setupid </b>: Set-up Id of corresponding bead <br>
<b>beadid</b>: Bead Id of first Set-up <br>
<b>c</b>: distance between two corresponding beads (c for color)  <br>
<b>c-norm</b>: distance divided by the highest distance in dataset (normalized)<br>
<b>x,y,z</b>: coordinates of corresponding bead from first Set-up for spatial plotting


### example how to use functions 

#define two lists: first: x and second:y <br>
x = ['24','25','26','27']<br>
y = ['29', '30', '31','32','33','34','35']<br>

<b>#step 1</b> read in initial coordinates with function <br>
read_in_initial_coordinates(first-list, second-list, name of beads for path, path to files,)<br>
<b>#step 2</b> extract all transformationmatrices <br>
read_in_transformation_matrix(path to files, first-list,second-list)<br>
<b>#step 3</b> transform intial coordinates with tranformation matrices into global coordinates <br>
transform_inital_to_global_coordinates(output from step 2, first-list, second-list, output from step1)<br>
<b>#step 4</b> read in corresponding bead pairs <br>
read_in_corresponding_beads(first-list, name of beads for path, path)<br>
<b>#step 5</b> calculate distance and safe dataframe + extra list of coordinates to make plotting easier
calculate_error(first-list,second-list, output from step 3, output from step 4)<br>

### limitations of method 

By only comparing one side to the three other sides, we are not comparing all possible combinations. <br>
We decided against it, to avoid any corresponding beads being counted double and to keep the computational effort low. <br>
Also, this comparison is sufficient to test if the sample is distored or not. <br>

In [ ]:
import numpy as np
import math 
import csv
#for reading in xml file 
from bs4 import BeautifulSoup
import pandas as pd

#read in all initial coordinates of all the bead-ids 
# creates nested dict: first key is set_up id
# value to setup id is dict of key is bead id and value are coordinates 
# input: list of both sides you want to compare and name of the points of interest (beads,nuclei), path to files 
# ouput combines both list of setupids, list_of_initial_beads_coordinates

def read_in_initial_coordinates(list_of_setup_ids1,list_of_setup_ids2,name,path):
    list_of_initial_beads_coordinates = {}
    # open files with build path from input 
    for number in list_of_setup_ids1: 
        with open(path + '/tpId_0_viewSetupId_' + number + '.'+ name +'.ip.txt') as file: 
            test = {}
            #create csv reader to itterate over rows 
            reader = csv.reader(file, delimiter = '\t')
            for row in reader: 
                # skip first row
                if row[0] == 'id':
                    continue
                #store coordinates as floats and in a dict already 
                else:
                    cor = [float(row[1]), float(row[2]), float(row[3]),1.0]
                    test[row[0]] = cor
            #create nested dict structure
            list_of_initial_beads_coordinates[number] = test
    # same but for second side   
    for number in list_of_setup_ids2: 
        with open(path + '/tpId_0_viewSetupId_' + number + '.'+ name +'.ip.txt') as file: 
            test = {}
            reader = csv.reader(file, delimiter = '\t')
            for row in reader: 
                if row[0] == 'id':
                    continue 
                else:
                    cor = [float(row[1]), float(row[2]), float(row[3]),1.0]
                    test[row[0]] = cor
            list_of_initial_beads_coordinates[number] = test
    return list_of_initial_beads_coordinates

# uses xml file to read in tms, creates a dict with setupids as keys and value is a list of tm matrixes
# input: path to xml file, list of both sides for setup ids 
#output all_tm 


def read_in_transformation_matrix(path_dataset,list_of_setup_ids1,list_of_setup_ids2): 
    #read in xml file for later converting it in a dataframe in beatifulsoup
    with open(path_dataset, 'r') as f:
        data = f.read()
    #create beautiful soup dataframe
    bs_data = BeautifulSoup(data, 'xml')

    all_tm = {}
    #searches tms for each setupid
    for number in list_of_setup_ids1: 
        #finds all the registrationmatrixes and 0 is only the registration matrixes, contents give you a list of tms 
        b_view_registrations = bs_data.find_all(setup=number)[0].contents
        #create empty list to append the finished tms 
        transformation_list = []
        #iterate over list that content gave you, there are still some "\n" stored that need to be skipped
        for elem in b_view_registrations:
            # skip "\n"
            if elem == "\n":
                continue
            else:
                #find in taq affine to get the transformation matrix
                a = elem.find("affine").contents
                # split all numbers into an array 
                a = np.asarray(a[0].split(' '), dtype = float)
                #add last row that is needed for full 4 by 4 tm 
                a = np.append(a,[0.0,0.0,0.0,1.0])
                # reshape array ionto 4 by 4 matrix 
                a = np.reshape(a,(4,4))
                # add tm to full tm list
                transformation_list.append(a)
        # store full tm list in dict with setupid as key 
        all_tm[number] = transformation_list
    #repeat for second side to have all tms
    for number in list_of_setup_ids2: 
        b_view_registrations = bs_data.find_all(setup=number)[0].contents
        transformation_list = []
        for elem in b_view_registrations:
            if elem == "\n":
                continue
            else:
                a = elem.find("affine").contents
                a = np.asarray(a[0].split(' '), dtype = float)
                a = np.append(a,[0.0,0.0,0.0,1.0])
                a = np.reshape(a,(4,4))
                transformation_list.append(a)
        all_tm[number] = transformation_list  
    return all_tm 

#transform the intital coordinates to global coordinates for all 
# input: list of all setup ids , plus their list of beadids with coordinates, plus list of tms for each set up id 
# get input from function: read_in_initial_doordinates and read_in_transformationmatrix 
#output list_of_global_beads_coordinates

def transform_inital_to_global_coordinates(tms,list_of_setup_ids1,list_of_setup_ids2, inital_cor_list):
    # transform the beads coordinates from inital to global bead coordinates
    #using the tms 
    list_of_global_beads_coordinates = {}
    #get inital coordinates from dict, output from inital bead coordinate function 
    for setup_id in list_of_setup_ids1: 
        #get dict stored in all inital_coordinate_dict
        list_of_ic = inital_cor_list[setup_id]
        local_dict = {}
        #calculate all the tms together in one tm 
        #this multiplys them in the right order first in read in (last to be applied to point) first in line for mulitplication
        # next is added to end of mulitplication 
        for i in range(len(tms[setup_id])):
            if i == 0:
                global_tm = tms[setup_id][i] @ tms[setup_id][i+1]
            elif i == 1:
                continue
            elif i >= 1:
                global_tm = global_tm @ tms[setup_id][i]
        # goes through bead_ids in list from setup_id
        for ids in list_of_ic: 
            #multiplys all tms for this setup to the inital coordinates for transformation 
            global_cor = global_tm @ list_of_ic[ids]
            #stores it in new global dict 
            local_dict[ids] = global_cor
        #adds dict of bead ids to dict with setup id as key
        list_of_global_beads_coordinates[setup_id] = local_dict

    # repeat for second side 
    for setup_id in list_of_setup_ids2: 
        list_of_ic = inital_cor_list[setup_id]
        local_dict = {}
        for i in range(len(tms[setup_id])):
            if i == 0:
                global_tm = tms[setup_id][i] @ tms[setup_id][i+1]
            elif i == 1:
                continue
            elif i >= 1:
                global_tm = global_tm @ tms[setup_id][i]
        for ids in list_of_ic: 
            global_cor = global_tm @ list_of_ic[ids]
            local_dict[ids] = global_cor
        list_of_global_beads_coordinates[setup_id] = local_dict
    return list_of_global_beads_coordinates



#opens the txt files that you want it to open given the list of set-up-ids, creates a  nested dict first key is first id
# key in value of first key is corresponding id, value is list of corresponding ids 
# input: the list of the set_up_ids from side1 you want to compare, path of where txt files are 

def read_in_corresponding_beads(list_of_setup_ids1,name, path):
    #opens the txt files that you want it to open given the list of set-up-ids, creates a  nested dict first key is first id
    # key in value of first key is corresponding id, value is list of corresponding ids 
    list_of_corresponding_beads = {}
    #read in each set up file one after one, first store all data in file_data, every row as an array
    for setup_id in list_of_setup_ids1:
        file_data = []
        setup_list = {}
        with open(path + '/tpId_0_viewSetupId_'+ setup_id +'.'+ name +'.corr.txt') as file:
            reader = csv.reader(file, delimiter = '\t')
            for row in reader:
                #get rid of first line 
                if row[0] == 'id':
                    continue 
                #get info needed from file 0 is set up id of corresponding, 1 is beadId of side x and 2 is beadID of side y
                #creates the dict with corresponding id as keys, later filled with the beadID pairs 
                else:
                    file_data.append([row[2],row[0],row[4]])
                    setup_list[row[2]] = []
            #fill setup_list with beadID pairs but seperate the beadIDs to the right setupIDs
            for key in setup_list:
                for elem in file_data:
                    if elem[0] == key:
                        id_pairs = setup_list[key]
                        id_pairs.append([elem[1],elem[2]])
                        setup_list[key] = id_pairs
                    else:
                        continue
            # save dict just created in overall output to generate nested dict structure 
            list_of_corresponding_beads[setup_id] = setup_list
            
    return list_of_corresponding_beads

# function for final setp of calculating the pixel distance between cooresponding beads 
# creates a list of coordinates corresponding to the first side and a list with their respective error
#beadids are lost, buy could be stored ? 
#NOT checked for points that have more corresponding beads in more setups 
# maybe additional outputs?  this is first step to use for napari points layer 
#input: list of both setup ids of both sides you want to compare, dict of the global coordinates from function 
# transform_inital_to_global_coordinates and list of corresponding bead ids from function read_in_corresponding_beads 
#output two things output_global_bead_coordinates_sidea,output_px_error_for_each_bead

def calculate_error(list_of_setup_ids1, list_of_setup_ids2, list_of_global_cord, list_of_corresponding_b):
    # calculate distance between corresponding beads 
    # create output in a form that you have coordinates for beads from first setup (comparing side a to side b) 
    # plus distance aka pixel error to corresponding beads 
    # get in a format that napari can read in 
    #create two output formats (list)
    output_global_bead_coordinates_sidea = []
    output_px_error_for_each_bead = []
    output_poi_id = []
    output_first_setup_id_for_df = []
    output_corresponding_setupid_for_df = []
    #only do this for first side 
    for setup_id in list_of_setup_ids1: 
        #get corresponding setups dict from function output read in corresponding beads 
        list_of_cor_beads = list_of_corresponding_b[setup_id]
        # if more corresponing setups then one by one being calculated 
        for key in list_of_cor_beads:
            #check if setup is in list of setupids we want to compare 
            if key in list_of_setup_ids2:
                #get list of beadid pairs 
                list_of_pair_specific_corresponding_beads = list_of_cor_beads[key]
                #go through list of pairs and calculate distance
                for pair in list_of_pair_specific_corresponding_beads:
                    #save first coordinate for setupid from side 1
                    cor1 = list_of_global_cord[setup_id][pair[0]]
                    # save second coordinate from corresponding setupid 
                    cor2 = list_of_global_cord[key][pair[1]]
                    #get coordinates from setupid1 in list without the last 1 
                    output_global_bead_coordinates_sidea.append(np.delete(cor1,3))
                    # add the distance calculated with the math module 
                    output_px_error_for_each_bead.append(math.dist(cor1,cor2))
                    #append beadid of each setup and append setupid for dataframe 
                    output_poi_id.append(pair[1])
                    output_corresponding_setupid_for_df.append(key)
                    output_first_setup_id_for_df.append(setup_id)
            else:
                continue
    #normalize all colors to maximal error in error output
    color = []
    for elem in output_px_error_for_each_bead:
        color.append(elem / max(output_px_error_for_each_bead))
    point_prop = {'setupid': output_first_setup_id_for_df, 'corresponding_setupid': output_corresponding_setupid_for_df, 'beadid': output_poi_id, 'c': output_px_error_for_each_bead, 'c-norm': color}
    a = []
    b = []
    c = []
    for elem in output_global_bead_coordinates_sidea:
        a.append(elem[0])
        b.append(elem[1])
        c.append(elem[2])
    point_prop['x'] = a
    point_prop['y'] = b
    point_prop['z'] = c
    df = pd.DataFrame(point_prop)
    
    return df, output_global_bead_coordinates_sidea
